In [48]:
import os
import numpy as np
import pandas as pd
import scipy.sparse as ssp
from functools import lru_cache
from tqdm import tqdm, trange
import time
import matplotlib.pyplot as plt
import cudf 

In [49]:
def flatten_sessions(sessions_df : pd.DataFrame):
    sess_id_list = []
    product_list = []
    in_hist = []
    for idx, row in tqdm(sessions_df.iterrows(), total=sessions_df.shape[0]):
        sess_id = idx
        prev_items = eval(row['prev_items'].replace(' ', ','))
        for prev_item in prev_items:
            sess_id_list.append(sess_id)
            product_list.append(prev_item)
            in_hist.append(1)
    return pd.DataFrame({'sess_id' : sess_id_list, 'product' : product_list, 'in_hist' : in_hist})

In [50]:
def cast_dtype(df : pd.DataFrame):
    for k in df.columns:
        dt = type(df[k].iloc[0])
        if 'float' in str(dt):
            df[k] = df[k].astype('float32')
        elif 'int' in str(dt):
            df[k] = df[k].astype('int32')
        elif dt == list:
            dt_ = type(df.iloc[0][k][0])
            if 'float' in str(dt_):
                df[k] = df[k].apply(lambda x : np.array(x, dtype=np.float32))
            elif 'int' in str(dt_):
                df[k] = df[k].apply(lambda x : np.array(x, dtype=np.int32))

# Valid Data

In [4]:
valid_candidates_2 = pd.read_parquet('/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/XGBoost/candidates/merged_candidates_2.parquet')
merged_candidates_feature = pd.read_parquet('/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/XGBoost/candidates/merged_candidates_feature.parquet')
valid_sessions = pd.read_csv('/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/data_for_recstudio/task1_data/task13_4_task1_valid_sessions.csv')

In [6]:
valid_candidates_2 = valid_candidates_2.sort_values(by=['sess_id', 'product'])
valid_candidates_2.reset_index(drop=True, inplace=True)

In [7]:
valid_sessions_flattened = flatten_sessions(valid_sessions)

100%|██████████| 361581/361581 [00:18<00:00, 19644.89it/s]


## filter candidates

In [8]:
valid_sessions_flattened_g = cudf.from_pandas(valid_sessions_flattened)
valid_candidates_g = cudf.from_pandas(valid_candidates_2)

In [9]:
valid_candidates_hist_g = valid_candidates_g.merge(valid_sessions_flattened_g, how='left', left_on=['sess_id', 'product'], right_on=['sess_id', 'product'])
valid_candidates_hist_g = valid_candidates_hist_g.sort_values(by=['sess_id', 'product'])
valid_candidates_hist_g.reset_index(drop=True, inplace=True)
valid_candidates_hist = valid_candidates_hist_g.to_pandas()
valid_candidates_hist['in_hist'] = valid_candidates_hist['in_hist'].fillna(0)

In [10]:
valid_candidates_filtered = valid_candidates_hist[valid_candidates_hist['in_hist'] != 1]
valid_candidates_filtered.reset_index(drop=True, inplace=True)
valid_candidates_filtered.drop(columns=['in_hist'], inplace=True)

/tmp/ipykernel_477530/2576615009.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_candidates_filtered.drop(columns=['in_hist'], inplace=True)


In [46]:
cast_dtype(valid_candidates_filtered)
valid_candidates_filtered.to_parquet('/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/XGBoost/candidates/merged_candidates_no_hist.parquet')

/tmp/ipykernel_477530/3837044860.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[k] = df[k].astype('int32')
/tmp/ipykernel_477530/3837044860.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[k] = df[k].astype('float32')


In [21]:
del valid_sessions_flattened_g
del valid_candidates_g
del valid_candidates_hist_g

In [11]:
valid_candidates_hist

,sess_id,sess_locale,product,target,in_hist
0,0,UK,B000OPPVCS,0.0,0.0
1,0,UK,B000V599Y2,0.0,0.0
2,0,UK,B0018HH444,0.0,0.0
3,0,UK,B0079JI4DU,0.0,0.0
4,0,UK,B0079JI4EY,0.0,0.0
...,...,...,...,...,...
85566178,361580,DE,B0BB7XV97M,0.0,0.0
85566179,361580,DE,B0BB7YSRBX,0.0,0.0
85566180,361580,DE,B0BB7ZMGY8,0.0,0.0
85566181,361580,DE,B0BD4CP7N3,0.0,0.0


In [45]:
valid_candidates_filtered

,sess_id,sess_locale,product,target
0,0,UK,B000OPPVCS,0.0
1,0,UK,B000V599Y2,0.0
2,0,UK,B0018HH444,0.0
3,0,UK,B0079JI4DU,0.0
4,0,UK,B0079JI4EY,0.0
...,...,...,...,...
84407334,361580,DE,B0BB7XV97M,0.0
84407335,361580,DE,B0BB7YSRBX,0.0
84407336,361580,DE,B0BB7ZMGY8,0.0
84407337,361580,DE,B0BD4CP7N3,0.0


In [47]:
valid_candidates_filtered.query('sess_id == 2').query('product == "B07VYSSRL7"')

,sess_id,sess_locale,product,target


In [30]:
valid_candidates_hist.query('sess_id == 2').query('product == "B07VYSSRL7"')

,sess_id,sess_locale,product,target,in_hist
560,2,UK,B07VYSSRL7,0.0,1.0


In [15]:
valid_sessions

,prev_items,next_item,locale
0,['B09VSN9GLS' 'B09VSG9DCG' 'B0BJ5L1ZPH' 'B09VS...,B06XG1LZ6Z,UK
1,['B00390YWXE' 'B00390YWXE' 'B09WM9W6WQ'],B01MSUI4FE,JP
2,['B01BM9V6H8' 'B01MG55XDR' 'B07VYSSRL7'],B01M6625ME,UK
3,['B092ZG24S7' 'B09BNHWWZM' 'B08CB1WG5M' '17880...,0241558573,UK
4,['B0B6NY5RM8' 'B09BJGBBBR'],B09BJF6N8K,JP
...,...,...,...
361576,['B08HH6L4PB' 'B08L8N8HDR'],B00TS5UXGY,UK
361577,['B08X4L1KLZ' 'B09BBX1T4S' 'B09D76FT9D'],B09BCM5NL1,JP
361578,['B0098G6L3M' 'B00ELRLP3O' 'B00PLXGK82' 'B09GS...,B0BC38GHB4,DE
361579,['B07Q2CNLY3' 'B07Q2CNLY3' 'B07BR7DZWN' 'B07Q2...,B08H8SYLMQ,DE


## filter candidate feature

In [8]:
valid_sessions_flattened_g = cudf.from_pandas(valid_sessions_flattened)
merged_candidates_g = cudf.from_pandas(merged_candidates)

In [11]:
merged_candidates_hist_g = merged_candidates_g.merge(valid_sessions_flattened_g, how='left', left_on=['sess_id', 'product'], right_on=['sess_id', 'product'])
merged_candidates_hist_g = merged_candidates_hist_g.sort_values(by=['sess_id', 'product'])
merged_candidates_hist_g.reset_index(drop=True, inplace=True)
merged_candidates_hist = merged_candidates_hist_g.to_pandas()
merged_candidates_hist['in_hist'] = merged_candidates_hist['in_hist'].fillna(0)

In [18]:
merged_candidates_feature_filtered = merged_candidates_feature[merged_candidates_hist['in_hist'] != 1]
merged_candidates_feature_filtered.reset_index(drop=True, inplace=True)

/tmp/ipykernel_704990/1748364940.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  merged_candidates_feature_filtered = merged_candidates_feature[merged_candidates_hist['in_hist'] != 1]


In [23]:
cast_dtype(merged_candidates_feature_filtered)
merged_candidates_feature_filtered.to_parquet('/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/XGBoost/candidates/merged_candidates_feature_no_hist.parquet')

/tmp/ipykernel_704990/3837044860.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[k] = df[k].astype('int32')
/tmp/ipykernel_704990/3837044860.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[k] = df[k].astype('float32')


: 

In [17]:
merged_candidates_hist[merged_candidates_hist['in_hist'] != 1]

,sess_id,sess_locale,product,in_hist
12496,0,UK,B000OPPVCS,0.0
12497,0,UK,B000V599Y2,0.0
12498,0,UK,B0018HH444,0.0
12499,0,UK,B0079JI4DU,0.0
12509,0,UK,B0079JI4EY,0.0
...,...,...,...,...
85642721,361580,DE,B0BB7XV97M,0.0
85643975,361580,DE,B0BB7YSRBX,0.0
85643978,361580,DE,B0BB7ZMGY8,0.0
85643976,361580,DE,B0BD4CP7N3,0.0


In [13]:
del valid_sessions_flattened_g
del merged_candidates_g
del merged_candidates_hist_g

In [15]:
valid_candidates_2

,sess_id,sess_locale,product,target
0,0,UK,B000OPPVCS,0.0
1,0,UK,B000V599Y2,0.0
2,0,UK,B0018HH444,0.0
3,0,UK,B0079JI4DU,0.0
4,0,UK,B0079JI4EY,0.0
...,...,...,...,...
85365224,361580,DE,B0BB7XV97M,0.0
85365225,361580,DE,B0BB7YSRBX,0.0
85365226,361580,DE,B0BB7ZMGY8,0.0
85365227,361580,DE,B0BD4CP7N3,0.0


In [18]:
valid_candidates_filtered

,sess_id,sess_locale,product,target
0,0,UK,B000OPPVCS,0.0
1,0,UK,B000V599Y2,0.0
2,0,UK,B0018HH444,0.0
3,0,UK,B0079JI4DU,0.0
4,0,UK,B0079JI4EY,0.0
...,...,...,...,...
84209438,361580,DE,B0B6QWY8Q7,0.0
84209439,361580,DE,B0BB7YSRBX,0.0
84209440,361580,DE,B0BB7ZMGY8,0.0
84209441,361580,DE,B0BD4CP7N3,0.0


In [21]:
merged_candidates_feature_filtered

,sess_id,sess_locale,product,target,sasrec_normalized_scores,roberta_normalized_scores,co_graph_normalized_counts,co_graph_normalized_counts_0,co_graph_normalized_counts_1,co_graph_normalized_counts_2,product_freq,gru4rec_scores,sess_avg_price,product_price,sasrec_scores,sasrec_scores_2,sasrec_normalized_scores_2
0,0,UK,B000OPPVCS,0.0,0.000000,0.000000,0.002748,0.001718,0.004051,0.003190,104,6.484859,7.388571,7.280000,8.668667,11.972421,1.994129e-04
1,0,UK,B000V599Y2,0.0,0.000298,0.000000,0.000000,0.000000,0.000000,0.000000,37,4.342063,7.388571,5.200000,11.681057,13.152878,6.492611e-04
2,0,UK,B0018HH444,0.0,0.000000,0.000000,0.002061,0.001718,0.004051,0.001595,7,3.220763,7.388571,15.800000,4.629130,5.606023,3.426590e-07
3,0,UK,B0079JI4DU,0.0,0.000000,0.000000,0.002404,0.001718,0.002026,0.003190,67,0.000000,7.388571,22.097065,0.000000,0.000000,1.259497e-09
4,0,UK,B0079JI4EY,0.0,0.000000,0.000000,0.002748,0.001718,0.004051,0.003190,77,0.000000,7.388571,22.097065,0.000000,0.000000,1.259497e-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84291540,361580,DE,B0BB7XV97M,0.0,0.000016,0.000000,0.000000,0.000000,0.000000,0.000000,56,9.268379,32.424000,47.990002,9.548986,9.117821,1.026431e-05
84291541,361580,DE,B0BB7YSRBX,0.0,0.000019,0.000000,0.000000,0.000000,0.000000,0.000000,58,7.047796,32.424000,43.990002,9.711373,9.163816,1.074745e-05
84291542,361580,DE,B0BB7ZMGY8,0.0,0.000055,0.000000,0.000000,0.000000,0.000000,0.000000,452,9.359167,32.424000,41.990002,10.757690,11.256460,8.712224e-05
84291543,361580,DE,B0BD4CP7N3,0.0,0.000000,0.006677,0.000000,0.000000,0.000000,0.000000,1,-0.593306,32.424000,24.990000,-5.042848,-3.778687,2.573046e-11


In [14]:
valid_sessions_flattened

,sess_id,product,in_hist
0,0,B09VSN9GLS,1
1,0,B09VSG9DCG,1
2,0,B0BJ5L1ZPH,1
3,0,B09VSN9GLS,1
4,0,B0BJ6V797Y,1
...,...,...,...
1520826,361580,B07N6J2M3K,1
1520827,361580,B081YZVG5L,1
1520828,361580,B011XK46F0,1
1520829,361580,B08427PFR5,1


# Test Data

In [51]:
merged_candidates_feature = pd.read_parquet('/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/XGBoost/candidates/merged_candidates_feature_test.parquet')
test_candidates = pd.read_parquet('/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/XGBoost/candidates/merged_candidates_test.parquet')
test_sessions = pd.read_csv('/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/raw_data/sessions_test_task1.csv')

In [52]:
test_candidates = test_candidates.sort_values(by=['sess_id', 'product'])
test_candidates.reset_index(drop=True, inplace=True)

In [53]:
test_sessions_flattened = flatten_sessions(test_sessions)

100%|██████████| 316971/316971 [00:17<00:00, 18535.82it/s]


## filter candidates 

In [54]:
test_sessions_flattened_g = cudf.from_pandas(test_sessions_flattened)
test_candidates_g = cudf.from_pandas(test_candidates)

In [55]:
test_candidates_hist_g = test_candidates_g.merge(test_sessions_flattened_g, how='left', left_on=['sess_id', 'product'], right_on=['sess_id', 'product'])
test_candidates_hist_g = test_candidates_hist_g.sort_values(by=['sess_id', 'product'])
test_candidates_hist_g.reset_index(drop=True, inplace=True)
test_candidates_hist = test_candidates_hist_g.to_pandas()
test_candidates_hist['in_hist'] = test_candidates_hist['in_hist'].fillna(0)

In [56]:
test_candidates_filtered = test_candidates_hist[test_candidates_hist['in_hist'] != 1]
test_candidates_filtered.drop(columns=['in_hist'], inplace=True)
test_candidates_filtered.reset_index(drop=True, inplace=True)

/tmp/ipykernel_477530/839736589.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_candidates_filtered.drop(columns=['in_hist'], inplace=True)


In [65]:
cast_dtype(test_candidates_filtered)
test_candidates_filtered.to_parquet('/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/XGBoost/candidates/merged_candidates_test_no_hist.parquet')

/tmp/ipykernel_477530/3837044860.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[k] = df[k].astype('int32')


: 

In [58]:
test_sessions

,prev_items,locale
0,['B08V12CT4C' 'B08V1KXBQD' 'B01BVG1XJS' 'B09VC...,DE
1,['B00R9R5ND6' 'B00R9RZ9ZS' 'B00R9RZ9ZS'],DE
2,['B07YSRXJD3' 'B07G7Q5N6G' 'B08C9Q7QVK' 'B07G7...,DE
3,['B08KQBYV43' '3955350843' '3955350843' '39553...,DE
4,['B09FPTCWMC' 'B09FPTQP68' 'B08HMRY8NG' 'B08TB...,DE
...,...,...
316966,['B077SZ2C3Y' 'B0B14M3VZX'],UK
316967,['B08KFHDPY9' 'B0851KTSRZ' 'B08KFHDPY9' 'B0851...,UK
316968,['B07PY1N81F' 'B07Q1Z8SQN' 'B07PY1N81F' 'B07Q1...,UK
316969,['B01MCQMORK' 'B09JYZ325W'],UK


In [63]:
test_candidates_filtered.query('sess_id==1').query("product=='B00R9RZ9ZS'")

,sess_id,sess_locale,product


In [64]:
test_candidates_filtered

,sess_id,sess_locale,product
0,0,DE,4088833651
1,0,DE,B000H6W2GW
2,0,DE,B000JG2RAG
3,0,DE,B000RYSOUW
4,0,DE,B000UGZVQM
...,...,...,...
69428426,316970,UK,B0BJCTH4NH
69428427,316970,UK,B0BJTQQWLG
69428428,316970,UK,B0BJV3RL4H
69428429,316970,UK,B0BK7SPC84


In [27]:
test_candidates_g

,sess_id,sess_locale,product
0,0,DE,4088833651
1,0,DE,B000H6W2GW
2,0,DE,B000JG2RAG
3,0,DE,B000RYSOUW
4,0,DE,B000UGZVQM
...,...,...,...
70320668,316970,UK,B0BJCTH4NH
70320669,316970,UK,B0BJTQQWLG
70320670,316970,UK,B0BJV3RL4H
70320671,316970,UK,B0BK7SPC84


## filter candidates feature

In [8]:
test_sessions_flattened_g = cudf.from_pandas(test_sessions_flattened)
merged_candidates_g = cudf.from_pandas(merged_candidates)

In [9]:
merged_candidates_hist_g = merged_candidates_g.merge(test_sessions_flattened_g, how='left', left_on=['sess_id', 'product'], right_on=['sess_id', 'product'])
merged_candidates_hist_g = merged_candidates_hist_g.sort_values(by=['sess_id', 'product'])
merged_candidates_hist = merged_candidates_hist_g.to_pandas()
merged_candidates_hist['in_hist'] = merged_candidates_hist['in_hist'].fillna(0)

In [12]:
del merged_candidates_hist_g
del test_sessions_flattened_g
del merged_candidates_g

In [14]:
merged_candidates_feature_filtered = merged_candidates_feature[merged_candidates_hist['in_hist'] != 1]
merged_candidates_feature_filtered.reset_index(drop=True, inplace=True)

/tmp/ipykernel_799600/3900097689.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  merged_candidates_feature_filtered = merged_candidates_feature[merged_candidates_hist['in_hist'] != 1]


In [16]:
cast_dtype(merged_candidates_feature_filtered)
merged_candidates_feature_filtered.to_parquet('/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/XGBoost/candidates/merged_candidates_feature_test_no_hist.parquet')

/tmp/ipykernel_799600/3837044860.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[k] = df[k].astype('int32')
/tmp/ipykernel_799600/3837044860.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[k] = df[k].astype('float32')


In [50]:
test_candidates_merged['in_hist'].sum() / 316971

3.465111950304602

In [45]:
test_sessions_flattened.shape[0] / 316971

4.26263601402021

In [15]:
merged_candidates_feature_filtered

,sess_id,sess_locale,product,sasrec_normalized_scores,roberta_normalized_scores,co_graph_normalized_counts,co_graph_normalized_counts_0,co_graph_normalized_counts_1,co_graph_normalized_counts_2,product_freq,gru4rec_scores,sess_avg_price,product_price,sasrec_scores,sasrec_scores_2,sasrec_normalized_scores_2
0,0,DE,4088833651,0.000000,0.000000,0.0,0.0,0.0,0.0,828,0.000000,25.195269,36.761604,0.000000,0.000000,1.278961e-09
1,0,DE,B000H6W2GW,0.000000,0.000000,0.0,0.0,0.0,0.0,875,0.000000,25.195269,36.761604,0.000000,0.000000,1.278961e-09
2,0,DE,B000JG2RAG,0.000000,0.005628,0.0,0.0,0.0,0.0,24,8.104032,25.195269,23.190001,7.664529,7.665308,2.728088e-06
3,0,DE,B000RYSOUW,0.000000,0.006545,0.0,0.0,0.0,0.0,5,-2.857798,25.195269,6.900000,-4.490097,-2.951060,6.686956e-11
4,0,DE,B000UGZVQM,0.000000,0.006410,0.0,0.0,0.0,0.0,4,4.688567,25.195269,21.990000,3.275950,3.977920,6.830395e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69307593,316970,UK,B0BJCTH4NH,0.000013,0.016727,0.0,0.0,0.0,0.0,74,10.629994,16.950001,5.800000,9.252855,11.327528,8.919876e-05
69307594,316970,UK,B0BJTQQWLG,0.000000,0.008232,0.0,0.0,0.0,0.0,6,6.052083,16.950001,9.880000,5.499319,5.604142,2.915573e-07
69307595,316970,UK,B0BJV3RL4H,0.000000,0.008672,0.0,0.0,0.0,0.0,7,7.667603,16.950001,22.097065,5.984032,9.146974,1.007758e-05
69307596,316970,UK,B0BK7SPC84,0.000000,0.020986,0.0,0.0,0.0,0.0,0,-6.356799,16.950001,5.960000,-13.903549,-10.383047,3.323348e-14


In [11]:
merged_candidates_g

,sess_id,sess_locale,product
0,0,DE,4088833651
1,0,DE,B000H6W2GW
2,0,DE,B000JG2RAG
3,0,DE,B000RYSOUW
4,0,DE,B000UGZVQM
...,...,...,...
70403346,316970,UK,B0BJCTH4NH
70403347,316970,UK,B0BJTQQWLG
70403348,316970,UK,B0BJV3RL4H
70403349,316970,UK,B0BK7SPC84


In [10]:
merged_candidates_hist

,sess_id,sess_locale,product,in_hist
928,0,DE,4088833651,0.0
929,0,DE,B000H6W2GW,0.0
930,0,DE,B000JG2RAG,0.0
931,0,DE,B000RYSOUW,0.0
932,0,DE,B000UGZVQM,0.0
...,...,...,...,...
70605569,316970,UK,B0BJCTH4NH,0.0
70605573,316970,UK,B0BJTQQWLG,0.0
70605570,316970,UK,B0BJV3RL4H,0.0
70605571,316970,UK,B0BK7SPC84,0.0


In [31]:
test_candidates.shape[0], test_candidates_filtered.shape[0]

(70320673, 69225510)

In [47]:
test_sessions_flattened

,sess_id,product,in_hist
0,0,B08V12CT4C,1
1,0,B08V1KXBQD,1
2,0,B01BVG1XJS,1
3,0,B09VC5PKN5,1
4,0,B09V7KG931,1
...,...,...,...
1351127,316969,B01MCQMORK,1
1351128,316969,B09JYZ325W,1
1351129,316970,B0B8JX92YJ,1
1351130,316970,B09TN4MP6V,1
